In [ ]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.5 MB/s eta 0:00:00


In [ ]:
pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
import cv2
import torch
import numpy as np
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
import timm
import torchmetrics
import copy
import torch.nn.functional as F


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


In [ ]:
ROOT_DIR = '/content/drive/MyDrive/Colab Notebooks/CS640/train_images_png'
TEST_DIR = '/content/drive/MyDrive/Colab Notebooks/CS640/test_images_png'

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS640/train.csv")
df["image_id"] = df["image_id"].apply(lambda x: f"{ROOT_DIR}/{x}.png")

# Label encoding
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [ ]:
# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_id']
        label = self.dataframe.iloc[idx]['label']
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image, label


In [ ]:
# Transforms
transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Dataloaders
train_dataset = CustomDataset(df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)


In [ ]:
class UBCModel(nn.Module):
    def __init__(self, model_name, num_classes, pretrained=True):
        super(UBCModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, features_only=True)

        # Use GeM Pooling
        self.gem = GeM()

        in_features = self.model.feature_info.channels()[-1]
        self.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        x = self.model(x)[-1]  # Get last stage feature map
        x = self.gem(x)  # Apply GeM pooling
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)  # Final fully connected layer
        return x

In [ ]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        return F.avg_pool2d(x.clamp(min=self.eps).pow(self.p), (x.size(-2), x.size(-1))).pow(1./self.p)


In [ ]:
# Model, Optimizer, and Criterion
model = UBCModel("tf_efficientnet_b0_ns", num_classes=5)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()


In [ ]:
# Validation data
val_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS640/test.csv")
val_df["image_id"] = val_df["image_id"].apply(lambda x: f"{TEST_DIR}/{x}.png")
if 'label' in val_df.columns:
    val_df['label'] = label_encoder.transform(val_df['label'])

val_dataset = CustomDataset(val_df, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
# Training and evaluation loops
for epoch in range(40):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 5:.3f}")
            running_loss = 0.0

print("Finished Training")


[1, 10] loss: 4.307
[2, 10] loss: 2.184
[3, 10] loss: 1.494
[4, 10] loss: 0.940
[5, 10] loss: 0.724
[6, 10] loss: 0.468
[7, 10] loss: 0.362
[8, 10] loss: 0.288
[9, 10] loss: 0.197
[10, 10] loss: 0.168
[11, 10] loss: 0.140
[12, 10] loss: 0.117
[13, 10] loss: 0.088
[14, 10] loss: 0.080
[15, 10] loss: 0.072
[16, 10] loss: 0.069
[17, 10] loss: 0.054
[18, 10] loss: 0.050
[19, 10] loss: 0.049
[20, 10] loss: 0.050
[21, 10] loss: 0.042
[22, 10] loss: 0.036
[23, 10] loss: 0.032
[24, 10] loss: 0.039
[25, 10] loss: 0.032
[26, 10] loss: 0.027
[27, 10] loss: 0.028
[28, 10] loss: 0.021
[29, 10] loss: 0.021
[30, 10] loss: 0.023
[31, 10] loss: 0.016
[32, 10] loss: 0.019
[33, 10] loss: 0.015
[34, 10] loss: 0.016
[35, 10] loss: 0.014
[36, 10] loss: 0.015
[37, 10] loss: 0.011
[38, 10] loss: 0.012
[39, 10] loss: 0.019
[40, 10] loss: 0.017
Finished Training


In [ ]:
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

def evaluate_model(model, val_loader, device):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    precision = precision_score(all_labels, all_preds, average='weighted')
    report = classification_report(all_labels, all_preds)

    return accuracy, f1, recall, precision, report

accuracy, f1, recall, precision, report = evaluate_model(model, val_loader, device)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print("\nClassification Report:\n", report)

Accuracy: 0.4722222222222222
F1 Score: 0.4722453291080742
Recall: 0.4722222222222222
Precision: 0.4757337838733188

Classification Report:
               precision    recall  f1-score   support

           0       0.38      0.45      0.41        20
           1       0.42      0.44      0.43        25
           2       0.65      0.62      0.64        45
           3       0.11      0.11      0.11         9
           4       0.33      0.22      0.27         9

    accuracy                           0.47       108
   macro avg       0.38      0.37      0.37       108
weighted avg       0.48      0.47      0.47       108

